In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from pathlib import Path
import os
from itertools import product
from scipy.signal import detrend

In [ ]:
# get_ipython().run_line_magic('matplotlib')
smallfont=11
bigfont=12
data = pd.read_excel(Path.cwd()/"CO2-K-EDGE-cleaned.xlsx")

In [ ]:
#Cleaning spectrum data to two columns and in txt format
# List of input files
input_files = [
    "H2O-aug-cc-pVDZ.spectrum",
    "H2O-aug-cc-pVTZ.spectrum",
    "H2O-aug-cc-pVQZ.spectrum",
    "H2O-cc-pVDZ.spectrum",
    "H2O-cc-pVTZ.spectrum",
    "H2O-cc-pVQZ.spectrum",
]

# Loop through each file and extract data
for input_file in input_files:
    output_file = f"extracted_{input_file}"  # Create output file name
    if os.path.exists(input_file):  # Check if file exists
        with open(input_file, "r") as infile, open(output_file, "w") as outfile:
            for line in infile:
                parts = line.split()
                if len(parts) >= 2 and parts[0].isdigit():  # Ensuring we only parse valid numerical rows
                    excitation_energy = parts[1]
                    osc_strength = parts[2]
                    outfile.write(f"{excitation_energy} {osc_strength}\n")
        print(f"Extracted data saved to {output_file}")
    else:
        print(f"File {input_file} not found.")


2 cols and 3 rows + exp

In [ ]:
#opening figure (plotting)
def gaussian(E, osc, sigma, grid):
    
    conv = []
    
    for Ei in x:
        tot=0
        for Ej, os in zip(E, osc):
            tot+=os*np.exp(-((((Ej-Ei)/sigma)**2)))
        conv.append(tot)
    return conv

In [ ]:

#importng data
fnames = ["aug-cc-pVDZ", "aug-cc-pVTZ", "aug-cc-pVQZ",
          "cc-pVDZ", "cc-pVTZ", "cc-pVQZ",
          "XAS-EXP-GAS"]

emin, emax = 525, 540
data_dict = {}
for fname in fnames:
    try:
        data = np.loadtxt(f"extracted_H2O-{fname}.spectrum")
    except:
        data = np.loadtxt(f"extracted_H2O-{fname}.spectrum", delimiter=',')
    data = data[data[:, 0] > emin]
    data = data[data[:, 0] < emax]
    data_dict[fname] = data


x = np.linspace(emin, emax, num=1000)
sigma=0.2
colors = ["red", "blue", "green", "red", "blue", "green", "green"]
fig, axs = plt.subplots(3, 2, figsize=(13.5, 10.5), layout="constrained")
shift = 4.41
scale = 1/ 2500 

for idx, fname, color in zip(range(len(fnames)), fnames, colors):
    data = data_dict[fname]
    gE = gaussian(data[:,0], data[:,1], sigma, x)

    if idx < 3:
        col_idx = 0
        axs[idx][col_idx].plot(x, gE, color=color, lw=2)#,   label=r'$\sigma$ = 0.8 eV')
        axs[idx][col_idx].set_title(f'Oxygen K-edge H2O(gas) x-ray ({fname})')
        axs[idx][col_idx].bar(data[:,0][:10], data[:,1][:10]*0.5, width=0.05, color='k', label='Sticks') #up to the row you want to plot the sticks


    if 2 < idx < 6:
        col_idx = 1
    
        axs[idx-3][col_idx].plot(x, gE, color=color, lw=2)#,   label=r'$\sigma$ = 0.8 eV')
        axs[idx-3][col_idx].set_title(f'Oxygen K-edge H2O(gas) x-ray ({fname})')
        axs[idx-3][col_idx].bar(data[:,0][:10], data[:,1][:10]*0.5, width=0.05, color='k', label='Sticks') #up to the row you want to plot the sticks

    if idx == 6:
        for i, j in product(range(3), range(2)): #for all 6 plots combinations [(0,0), (0, 1), (0, 2), (1, 1),(1, 2), (1, 3)]
            
            #axs[i][j].plot(x - shift, np.array(gE) * scale, color='k', lw=2, linestyle="--") #,   label=r'$\sigma$ = 0.8 eV')
            axs[i][j].plot(data_dict["XAS-EXP-GAS"][:,0] - shift,
                            (detrend(data_dict["XAS-EXP-GAS"][:,1]) + 34) * scale,
                            color='k', lw=2, linestyle="--")
            axs[i][j].set_xlabel('Energy / eV', fontsize=bigfont)
            axs[i][j].set_ylabel('Intensity / arb. u.', fontsize=bigfont)
            axs[i][j].set_xlim(524, 540)
            axs[i][j].set_ylim(0, 0.05)
            axs[i][j].spines[["right", "top"]].set_visible(False)
    

